In [1]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
)

In [2]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

## load_and_sample_petri_model

In [3]:
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling w/o an intervention
samples = load_and_sample_petri_model(ASKENET_PATH, num_samples, timepoints=timepoints)

# Save results
samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

In [4]:
from pyciemss.PetriNetODE.interfaces import load_petri_model

model = load_petri_model(ASKENET_PATH)
model

ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.024299999698996544, high: 0.02969999983906746),
	gamma = Uniform(low: 0.12600000202655792, high: 0.15399999916553497),
	pseudocount = 1.0
)

In [7]:
# Run sampling w/o an intervention
# intervention is an iterable (e.g. list) of tuples of the form (timepoint: float, parameter: str, value: float)
interventions = [(1.1, "beta", 100.0), (2.1, "gamma", 0.1)]

samples = load_and_sample_petri_model(
    ASKENET_PATH, num_samples, timepoints=timepoints, interventions=interventions
)

# Save results
samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/sample_results_w_interventions.csv"),
    index=False,
)

In [6]:
samples

,timepoint_id,sample_id,beta_param,gamma_param,Infected_sol,Recovered_sol,Susceptible_sol
0,0,0,0.028574,0.140698,1.000000,1.406979e-11,1000.000000
1,1,0,0.028574,0.140698,0.893908,1.330950e-01,999.972900
2,2,0,0.028574,0.140698,1.220722,2.777135e-01,999.501343
3,3,0,0.028574,0.140698,1.219600,4.525110e-01,999.328064
4,4,0,0.028574,0.140698,1.170759,6.206470e-01,999.208374
...,...,...,...,...,...,...,...
495,0,99,0.026700,0.138754,1.000000,1.387543e-11,1000.000000
496,1,99,0.026700,0.138754,0.893972,1.312609e-01,999.973694
497,2,99,0.026700,0.138754,1.222955,2.740130e-01,999.503052
498,3,99,0.026700,0.138754,1.224209,4.468783e-01,999.328857


## load_and_calibrate_and_sample_petri_model

In [7]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 37.94811773300171
iteration 25: loss = 34.9170196056366
iteration 50: loss = 32.69080424308777
iteration 75: loss = 32.41730046272278
iteration 100: loss = 32.44331431388855
iteration 125: loss = 32.692076683044434
iteration 150: loss = 32.61838483810425
iteration 175: loss = 32.56607246398926
iteration 200: loss = 33.014737367630005
iteration 225: loss = 32.58268332481384
iteration 250: loss = 32.89216685295105
iteration 275: loss = 31.961626768112183
iteration 300: loss = 32.6565682888031
